# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,0,2,0,4,1,1,0,1,1,0
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,1,3,1,0,0,1,2,0,0,0
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,3,0,2,1,1,0,1,0,1,0
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,FOODS_3_823_CA_4_validation,FOODS_3_823,FOODS_3,FOODS,CA_4,CA,1,1,1,4,...,3,1,2,2,0,1,1,3,3,0
12192,FOODS_3_824_CA_4_validation,FOODS_3_824,FOODS_3,FOODS,CA_4,CA,0,0,0,1,...,0,1,1,1,0,2,0,0,0,1
12193,FOODS_3_825_CA_4_validation,FOODS_3_825,FOODS_3,FOODS,CA_4,CA,2,3,2,1,...,0,2,0,1,3,1,1,0,2,0
12194,FOODS_3_826_CA_4_validation,FOODS_3_826,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,2,1,4,5,1,7,0,4


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Foods'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [7]:
len(sales_intermittent)

1301

In [8]:
len(sales_lumpy)

395

In [9]:
len(sales_erratic)

458

In [10]:
len(sales_smooth)

3594

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [11]:
sales_lumpy

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,3,1,1,2,0,2,2,1,4,1
1625,FOODS_1_015_CA_1_validation,FOODS_1_015,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,1,1,0,2,0,1,2,2,0
1637,FOODS_1_027_CA_1_validation,FOODS_1_027,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,5,4,2,3,1,0,2,1,3,3
1638,FOODS_1_028_CA_1_validation,FOODS_1_028,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,1,5,4,2,1,1,1,3,0,0
1640,FOODS_1_030_CA_1_validation,FOODS_1_030,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,2,0,0,1,1,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12147,FOODS_3_779_CA_4_validation,FOODS_3_779,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12155,FOODS_3_787_CA_4_validation,FOODS_3_787,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,0,2,2,1,0,1,0,0,3
12168,FOODS_3_800_CA_4_validation,FOODS_3_800,FOODS_3,FOODS,CA_4,CA,1,2,4,2,...,0,3,2,3,1,1,2,1,2,2
12183,FOODS_3_815_CA_4_validation,FOODS_3_815,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,0,1,0,2,0,0,0,0,0,0


# User-defined functions to calculate Metrics and Croston algorithm

In [12]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [13]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [14]:
sales_pattern = sales_lumpy.copy()

In [15]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [16]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [17]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [18]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [19]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [20]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [21]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [22]:
len(sample_list_filtered)

123

# Hyperparameter tuning for Croston

In [23]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [24]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: FOODS_3_465_CA_4
Progressing: 0.0 %
Currently Running: FOODS_3_713_CA_3
Progressing: 0.81 %
Currently Running: FOODS_1_211_CA_4
Progressing: 1.63 %
Currently Running: FOODS_3_567_CA_2
Progressing: 2.44 %
Currently Running: FOODS_2_048_CA_3
Progressing: 3.25 %
Currently Running: FOODS_1_030_CA_1
Progressing: 4.07 %
Currently Running: FOODS_2_251_CA_4
Progressing: 4.88 %
Currently Running: FOODS_3_556_CA_1
Progressing: 5.69 %
Currently Running: FOODS_3_267_CA_2
Progressing: 6.5 %
Currently Running: FOODS_3_360_CA_2
Progressing: 7.32 %
Currently Running: FOODS_3_167_CA_2
Progressing: 8.13 %
Currently Running: FOODS_3_806_CA_1
Progressing: 8.94 %
Currently Running: FOODS_3_082_CA_2
Progressing: 9.76 %
Currently Running: FOODS_3_660_CA_3
Progressing: 10.57 %
Currently Running: FOODS_3_293_CA_2
Progressing: 11.38 %
Currently Running: FOODS_3_664_CA_3
Progressing: 12.2 %
Currently Running: FOODS_3_056_CA_2
Progressing: 13.01 %
Currently Running: FOODS_1_075_CA_4
Progressing

In [25]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,FOODS_3_465_CA_4,9.0,4.800952,0.05
1,FOODS_3_465_CA_4,2.0,4.946481,0.05
2,FOODS_3_465_CA_4,5.0,4.842772,0.05
3,FOODS_3_465_CA_4,6.0,4.848389,0.05
4,FOODS_3_465_CA_4,1.0,4.890128,0.05
...,...,...,...,...
17707,FOODS_2_380_CA_1,24.0,30.383099,0.90
17708,FOODS_2_380_CA_1,29.0,24.689552,0.90
17709,FOODS_2_380_CA_1,24.0,28.565467,0.90
17710,FOODS_2_380_CA_1,20.0,24.456916,0.90


# Summay all metrics

In [26]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [27]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.06863596918333315 minutes


In [28]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_010_CA_2,0.05,0.765614,99.9986,185.6300,92.1600,0.874988,1.300660
1,FOODS_1_010_CA_2,0.10,0.765585,99.9947,182.1354,90.0374,0.874954,1.282158
2,FOODS_1_010_CA_2,0.15,0.765579,99.9939,177.7647,87.2453,0.874947,1.258549
3,FOODS_1_010_CA_2,0.20,0.765745,100.0156,172.5525,83.6951,0.875137,1.230382
4,FOODS_1_010_CA_2,0.25,0.766320,100.0908,166.7706,79.4393,0.875795,1.200324
...,...,...,...,...,...,...,...,...
2209,FOODS_3_806_CA_1,0.70,1.838749,94.7702,NaN,96.6868,6.041603,9.665242
2210,FOODS_3_806_CA_1,0.75,1.803660,92.9617,NaN,95.6000,5.926310,9.428977
2211,FOODS_3_806_CA_1,0.80,1.750365,90.2149,NaN,93.9543,5.751200,9.075108
2212,FOODS_3_806_CA_1,0.85,1.667537,85.9459,NaN,91.3920,5.479052,8.540947


# Check what products has unexpected metrics result

In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
558,FOODS_2_165_CA_1,0.05,inf,inf,200.0,NaN,0.415720,0.415720
559,FOODS_2_165_CA_1,0.10,inf,inf,200.0,NaN,0.488335,0.488335
560,FOODS_2_165_CA_1,0.15,inf,inf,200.0,NaN,0.582427,0.582427
561,FOODS_2_165_CA_1,0.20,inf,inf,200.0,NaN,0.705783,0.705783
562,FOODS_2_165_CA_1,0.25,inf,inf,200.0,NaN,0.869078,0.869078
...,...,...,...,...,...,...,...,...
1777,FOODS_3_556_CA_1,0.70,inf,inf,200.0,NaN,0.080632,0.080632
1778,FOODS_3_556_CA_1,0.75,inf,inf,200.0,NaN,0.076674,0.076674
1779,FOODS_3_556_CA_1,0.80,inf,inf,200.0,NaN,0.073179,0.073179
1780,FOODS_3_556_CA_1,0.85,inf,inf,200.0,NaN,0.070067,0.070067


In [30]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
558,FOODS_2_165_CA_1,0.05,inf,inf,200.0,NaN,0.415720,0.415720
559,FOODS_2_165_CA_1,0.10,inf,inf,200.0,NaN,0.488335,0.488335
560,FOODS_2_165_CA_1,0.15,inf,inf,200.0,NaN,0.582427,0.582427
561,FOODS_2_165_CA_1,0.20,inf,inf,200.0,NaN,0.705783,0.705783
562,FOODS_2_165_CA_1,0.25,inf,inf,200.0,NaN,0.869078,0.869078
...,...,...,...,...,...,...,...,...
1777,FOODS_3_556_CA_1,0.70,inf,inf,200.0,NaN,0.080632,0.080632
1778,FOODS_3_556_CA_1,0.75,inf,inf,200.0,NaN,0.076674,0.076674
1779,FOODS_3_556_CA_1,0.80,inf,inf,200.0,NaN,0.073179,0.073179
1780,FOODS_3_556_CA_1,0.85,inf,inf,200.0,NaN,0.070067,0.070067


In [31]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
504,FOODS_2_125_CA_4,0.05,0.453125,103.5714,NaN,100.0000,0.517857,1.414664
505,FOODS_2_125_CA_4,0.10,0.453125,103.5714,NaN,100.0000,0.517857,1.414664
506,FOODS_2_125_CA_4,0.15,0.453125,103.5714,NaN,100.0000,0.517857,1.414664
507,FOODS_2_125_CA_4,0.20,0.453125,103.5714,NaN,100.0000,0.517857,1.414664
508,FOODS_2_125_CA_4,0.25,0.453125,103.5714,NaN,100.0000,0.517857,1.414664
...,...,...,...,...,...,...,...,...
2209,FOODS_3_806_CA_1,0.70,1.838749,94.7702,NaN,96.6868,6.041603,9.665242
2210,FOODS_3_806_CA_1,0.75,1.803660,92.9617,NaN,95.6000,5.926310,9.428977
2211,FOODS_3_806_CA_1,0.80,1.750365,90.2149,NaN,93.9543,5.751200,9.075108
2212,FOODS_3_806_CA_1,0.85,1.667537,85.9459,NaN,91.3920,5.479052,8.540947


In [32]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
558,FOODS_2_165_CA_1,0.05,inf,inf,200.0,NaN,0.415720,0.415720
559,FOODS_2_165_CA_1,0.10,inf,inf,200.0,NaN,0.488335,0.488335
560,FOODS_2_165_CA_1,0.15,inf,inf,200.0,NaN,0.582427,0.582427
561,FOODS_2_165_CA_1,0.20,inf,inf,200.0,NaN,0.705783,0.705783
562,FOODS_2_165_CA_1,0.25,inf,inf,200.0,NaN,0.869078,0.869078
...,...,...,...,...,...,...,...,...
1777,FOODS_3_556_CA_1,0.70,inf,inf,200.0,NaN,0.080632,0.080632
1778,FOODS_3_556_CA_1,0.75,inf,inf,200.0,NaN,0.076674,0.076674
1779,FOODS_3_556_CA_1,0.80,inf,inf,200.0,NaN,0.073179,0.073179
1780,FOODS_3_556_CA_1,0.85,inf,inf,200.0,NaN,0.070067,0.070067


In [33]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [34]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [35]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [36]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 5.691056910569105 %
Percentage of unexpected values of WMAPE is: 5.691056910569105 %
Percentage of unexpected values of SMAPE is: 4.0650406504065035 %
Percentage of unexpected values of MAPE is: 5.691056910569105 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [37]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [38]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.301158,99.367075,95.388856,76.496063,5.899984,7.103355
0.10,1.226985,91.319605,89.061689,68.472618,5.549484,6.637693
0.15,1.233865,94.128362,86.419537,67.874676,5.328221,6.363850
0.20,1.228935,95.245214,84.669466,67.122343,5.117526,6.142019
0.25,1.208422,94.042778,83.560755,66.202349,4.950255,5.976262
0.30,1.183034,91.988584,82.701787,65.249988,4.810046,5.858297
0.35,1.162321,90.193258,82.221501,64.749013,4.692311,5.770180
0.40,1.145426,88.638131,81.931647,64.462887,4.602855,5.700981
0.45,1.132081,87.430153,81.854201,64.492333,4.556787,5.646031


In [39]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.7
The optimum Alpha based on Mean WMAPE is: 0.7
The optimum Alpha based on Mean SMAPE is: 0.55
The optimum Alpha based on Mean MAPE is: 0.4
The optimum Alpha based on Mean MAE is: 0.7
The optimum Alpha based on Mean RMSE is: 0.65


### Get MEDIAN metrics value of each Alpha

In [40]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.062117,72.15745,86.76505,61.24290,3.114458,3.851939
0.10,0.998497,67.98890,75.80255,55.38315,2.632921,3.440790
0.15,0.938767,66.25455,73.33200,56.46700,2.630586,3.414761
0.20,0.903654,65.49920,71.27225,57.10015,2.606581,3.143779
0.25,0.882766,64.06840,69.87680,57.02665,2.553123,3.132278
0.30,0.843319,62.67790,67.56535,57.09930,2.536494,3.178992
0.35,0.839414,62.89210,66.76190,58.07205,2.524637,3.249838
0.40,0.853301,61.06010,66.87810,57.81635,2.595742,3.278589
0.45,0.861317,62.01375,67.49110,57.26275,2.609716,3.329361


In [41]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.35
The optimum Alpha based on Median of WMAPE is: 0.4
The optimum Alpha based on Median of SMAPE is: 0.35
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.65
The optimum Alpha based on Median of RMSE is: 0.25


# Run Croston model after decide best paramaters

In [42]:
best_alpha = 0.4

In [43]:
dataframe_weekly

,wm_yr_wk,FOODS_1_005_CA_1,FOODS_1_008_CA_2,FOODS_1_010_CA_2,FOODS_1_010_CA_4,FOODS_1_015_CA_1,FOODS_1_015_CA_2,FOODS_1_015_CA_3,FOODS_1_022_CA_4,FOODS_1_027_CA_1,...,FOODS_3_800_CA_3,FOODS_3_800_CA_4,FOODS_3_806_CA_1,FOODS_3_806_CA_3,FOODS_3_815_CA_2,FOODS_3_815_CA_4,FOODS_3_816_CA_2,FOODS_3_824_CA_3,FOODS_3_825_CA_2,FOODS_3_827_CA_4
0,11101,18.0,0.0,4.0,2.0,0.0,1.0,5.0,0.0,9.0,...,99.0,5.0,3.0,2.0,13.0,0.0,0.0,3.0,4.0,0.0
1,11102,31.0,0.0,2.0,0.0,1.0,3.0,1.0,0.0,20.0,...,42.0,3.0,4.0,0.0,3.0,0.0,0.0,1.0,3.0,0.0
2,11103,5.0,0.0,2.0,7.0,0.0,1.0,0.0,3.0,7.0,...,45.0,5.0,6.0,0.0,5.0,0.0,0.0,0.0,7.0,0.0
3,11104,10.0,0.0,3.0,2.0,1.0,3.0,1.0,4.0,10.0,...,64.0,8.0,12.0,0.0,0.0,0.0,0.0,1.0,10.0,0.0
4,11105,4.0,0.0,4.0,3.0,8.0,1.0,0.0,6.0,7.0,...,32.0,1.0,7.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,12.0,0.0,0.0,2.0,0.0,0.0,3.0,1.0,13.0,...,57.0,8.0,11.0,12.0,0.0,0.0,3.0,6.0,6.0,0.0
269,11609,9.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,8.0,...,23.0,3.0,14.0,0.0,0.0,0.0,1.0,3.0,5.0,0.0
270,11610,13.0,0.0,3.0,0.0,1.0,2.0,2.0,0.0,17.0,...,46.0,8.0,23.0,4.0,2.0,0.0,2.0,8.0,6.0,0.0
271,11611,4.0,0.0,1.0,0.0,2.0,0.0,3.0,4.0,15.0,...,54.0,11.0,17.0,0.0,0.0,0.0,0.0,2.0,11.0,0.0


In [44]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [45]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [46]:
df_result_best_params

,Product,Actual Data,Forecast
0,FOODS_1_005_CA_1,4.0,11.461663
1,FOODS_1_005_CA_1,6.0,8.509919
2,FOODS_1_015_CA_1,2.0,0.518569
3,FOODS_1_015_CA_1,0.0,0.752233
4,FOODS_1_027_CA_1,15.0,14.541884
...,...,...,...
785,FOODS_3_800_CA_4,3.0,8.412745
786,FOODS_3_815_CA_4,0.0,2.399103
787,FOODS_3_815_CA_4,0.0,2.399103
788,FOODS_3_827_CA_4,0.0,0.000000


In [47]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [48]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_005_CA_1,2.492896,99.7158,65.5571,114.1868,4.985791,5.566692
1,FOODS_1_008_CA_2,NaN,NaN,NaN,NaN,0.000000,0.000000
2,FOODS_1_010_CA_2,inf,7.1736,7.4454,7.1736,0.071736,0.072349
3,FOODS_1_010_CA_4,2.133324,426.6648,144.9540,163.3324,2.133324,2.191135
4,FOODS_1_015_CA_1,0.558416,111.6832,158.8203,74.0715,1.116832,1.174839
...,...,...,...,...,...,...,...
390,FOODS_3_815_CA_4,inf,inf,200.0000,NaN,2.399103,2.399103
391,FOODS_3_816_CA_2,inf,inf,200.0000,NaN,3.953677,3.953677
392,FOODS_3_824_CA_3,1.691790,338.3579,142.6558,148.7709,3.383579,3.408108
393,FOODS_3_825_CA_2,0.672814,50.4611,53.2509,52.4488,4.036886,4.197333


In [49]:
df_result_final.isnull().sum()

Product      0
MASE        80
WMAPE       80
SMAPE       80
MAPE       158
MAE          0
RMSE         0
dtype: int64

In [50]:
df_result_final.MAPE.mean()

80.08438734177216

In [51]:
df_result_final.to_csv('Croston_Lumpy_Metrics.csv')